### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Cognitive_Processing/GSE202033'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Dataset indicates RNA expression data
is_gene_available = True

# Check availability of variables in sample characteristics dictionary
sample_characteristics = {
    0: ['brain region: Dorsolateral Prefrontal cortex (Brodmann area 46)'], 
    1: ['age: 71'], 
    2: ['Sex: Female'], 
    3: ['molecule subtype: nuclear RNA + genomic DNA']
}

# Row for age
age_row = 1

# Row for gender
gender_row = 2

# Determine row for 'Cognitive_Processing' trait (Placeholder, supposedly non-existent in current data)
trait_row = None  # No trait row available based on the provided data dictionary

# Convert age to continuous data
def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except (ValueError, IndexError):
        return None

# Convert gender to binary (female to 0, male to 1)
def convert_gender(value):
    val = value.split(': ')[1].strip().lower()
    if val == 'female':
        return 0
    elif val == 'male':
        return 1
    else:
        return None

# Save cohort information
save_cohort_info('GSE202033', './preprocessed/Cognitive_Processing/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction if age_row or gender_row is available
if age_row is not None or gender_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Cognitive_Processing', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Cognitive_Processing/trait_data/GSE202033.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
